# Imports

In [1]:
from components.ingestion.synthetic.event_synthetic import EventSynthetic

# Init Component

In [2]:
root_path = "/opt/dev"

general_config = f"{root_path}/pipe_configs/configurations/general_config.ini"
component_config = f"{root_path}/pipe_configs/configurations/event/event_synthetic.ini"

In [3]:
# Start Component
event_synthetic = EventSynthetic(general_config, component_config)
event_synthetic.spark

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/11/28 14:24:18 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


# Extract

# Transform

In [5]:
from core.data_objects.bronze.bronze_event_data_object import BronzeEventDataObject
import random

# MVP - Synthetic
spark = event_synthetic.spark
schema = event_synthetic.output_data_objects[BronzeEventDataObject.ID].SCHEMA

n_agents = event_synthetic.config.getint('Component.EventSynthetic', 'n_agents')
n_events = event_synthetic.config.getint('Component.EventSynthetic', 'n_events')
seed = event_synthetic.config.getint('Component.EventSynthetic', 'seed')

sdate = event_synthetic.config.getint('Component.EventSynthetic', 'dates_string')

nrows = n_agents * n_events


data = []
for _ in range(n_agents):
    
    # Empty dataframe
    # df = spark.sparkContext.emptyRDD().toDF(schema)

    # Random user_id
    # random.seed(seed)
    user_id = random.randbytes(32)

    for _ in range(n_events):
        # Random timestamp in path
        random_date = f"{sdate}T{random.randint(0, 23):02}:{random.randint(0, 59):02}:{random.randint(0, 59):02}"
        # random mcc
        random_mcc = random.randint(0, 999)
        # random cell_id
        cell_id = f"{random_mcc:03}" + "".join([str(random.randint(0, 9)) for _ in range(12)])

        # latitude
        latitude = None
        # longitude
        longitude = None
        # loc_error
        loc_error = None
        # Create row
        row = [user_id, random_date, random_mcc, cell_id, latitude, longitude, loc_error]
        # Append row to data
        data.append(row)


df = spark.createDataFrame(data, schema)
df.show()

+--------------------+-----------------+---+---------------+--------+---------+---------+
|             user_id|        timestamp|mcc|        cell_id|latitude|longitude|loc_error|
+--------------------+-----------------+---+---------------+--------+---------+---------+
|[51 29 ED F0 73 E...|20230101T21:10:55|701|701980550707634|    null|     null|     null|
|[51 29 ED F0 73 E...|20230101T12:42:22|108|108519696110357|    null|     null|     null|
|[51 29 ED F0 73 E...|20230101T01:52:21|423|423644015408110|    null|     null|     null|
|[A5 EA 32 F0 06 E...|20230101T14:16:59|300|300525335702358|    null|     null|     null|
|[A5 EA 32 F0 06 E...|20230101T17:56:19|702|702534189650107|    null|     null|     null|
|[A5 EA 32 F0 06 E...|20230101T12:00:38|968|968553688052984|    null|     null|     null|
+--------------------+-----------------+---+---------------+--------+---------+---------+



In [6]:
event_do = event_synthetic.output_data_objects[BronzeEventDataObject.ID]
sdate = event_synthetic.config.get('Component.EventSynthetic', 'dates_string')
path = f"{event_do.interface.path}/{sdate}"
print(path)

/opt/data/lakehouse/bronze/mno_events/20230101


In [9]:
event_do.df = df
event_do.write(path)

In [16]:
temp_output_path = "/opt/data/lakehouse/test_partition"
df.write.format("parquet").partitionBy(["user_id"]).mode("overwrite").save(temp_output_path)

# Load

In [44]:
import os
import random

bits = random.randbytes(32)
type(bits)

bytes

In [45]:
len(bin(int.from_bytes(bits))[2:])

256